In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("diff_data/cems_diffs_isorto.csv", index_col='DATE_UTC')

In [4]:
# Remove columns with damage data 
df = df[df.columns[~df.columns.str.endswith('ap2')]]
df = df[df.columns[~df.columns.str.endswith('dam')]]
df = df[df.columns[~df.columns.str.endswith('eas')]]
df.head()

,isorto,gload_mwh,so2_kg,nox_kg,pm25_kg,co2_kg
DATE_UTC,,,,,,
2006-01-01 04:00:00,CAISO,-457.19,-0.896751,-43.825605,-8.436821,-189575.356635
2006-01-01 04:00:00,ERCOT,-445.70,-1853.107025,-138.292036,-20.593100,-341468.969925
2006-01-01 04:00:00,ISONE,-142.25,-525.355244,4.911948,-13.607775,-64021.859820
2006-01-01 04:00:00,MISO,-169.00,-211.712705,-208.724441,-30.753571,-161730.220245
2006-01-01 04:00:00,NYISO,10.00,-180.070127,0.725747,-3.084429,10160.472000


In [5]:
# calculate MEF (assumed to be emissions / generation)
df['co2_mef'] = df['co2_kg']/df['gload_mwh']
df['so2_mef'] = df['so2_kg']/df['gload_mwh']
df['nox_mef'] = df['nox_kg']/df['gload_mwh']
df['pm25_mef'] = df['pm25_kg']/df['gload_mwh']
df = df[df.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]
df = df.dropna() 

In [6]:
df.co2_mef.describe()

count    7.360570e+05
mean     5.912107e+02
std      3.870106e+04
min     -1.800064e+07
25%      4.184159e+02
50%      5.570117e+02
75%      7.381082e+02
max      2.451540e+07
Name: co2_mef, dtype: float64

### Why are MEFs negative? 

In [27]:
df_pjm = df.loc[df['isorto'] == 'PJM']

# Keep only negative MEFs
mef_columns = ['co2_mef', 'so2_mef', 'nox_mef', 'pm25_mef']
df_pjm_neg = df_pjm[(df_pjm[mef_columns]<0).any(axis='columns')]
df_pjm_neg

,isorto,gload_mwh,so2_kg,nox_kg,pm25_kg,co2_kg,co2_mef,so2_mef,nox_mef,pm25_mef
DATE_UTC,,,,,,,,,,
2006-01-01 05:00:00,PJM,-4.00,-1804.548640,13.971541,78.743658,-6.214217e+04,15535.543125,451.137160,-3.492885,-19.685915
2006-01-01 06:00:00,PJM,702.00,-631.334293,705.651260,160.027434,2.559178e+05,364.555265,-0.899337,1.005201,0.227959
2006-01-01 11:00:00,PJM,-131.70,120.039948,211.556216,56.154752,-3.495475e+04,265.411885,-0.911465,-1.606349,-0.426384
2006-01-01 13:00:00,PJM,38.00,2451.936915,-202.981059,14.424242,3.674009e+04,966.844350,64.524656,-5.341607,0.379585
2006-01-02 03:00:00,PJM,-261.80,128.591971,-749.742670,-38.283207,-1.357103e+05,518.374103,-0.491184,2.863799,0.146231
2006-01-02 14:00:00,PJM,-62.45,555.491443,362.554725,-90.718500,-9.449239e+04,1513.088705,-8.894979,-5.805520,1.452658
2006-01-02 16:00:00,PJM,-82.84,-1425.488610,-1333.061529,67.222408,-2.197774e+05,2653.034279,17.207733,16.092003,-0.811473
2006-01-02 19:00:00,PJM,-580.80,4152.033751,934.626316,-140.250801,1.756927e+05,-302.501213,-7.148818,-1.609205,0.241479
2006-01-02 20:00:00,PJM,-453.67,-1949.864095,412.268408,-124.193627,-3.691944e+05,813.794956,4.297979,-0.908741,0.273753


In [14]:
# Get data from PJM data miner 
df_pjm_gen = pd.read_csv("pjm_data/pjm_gen.csv", index_col='datetime_beginning_utc')
df_pjm_gen.index = pd.to_datetime(df_pjm_gen.index)
del df_pjm_gen['datetime_beginning_ept']
del df_pjm_gen['is_renewable']

# keep only fossil fuel rows 
fossil_fuel = ['Coal', 'Gas', 'Oil']
df_pjm_gen = df_pjm_gen.loc[df_pjm_gen['fuel_type'].isin(fossil_fuel)]
df_pjm_gen.head()

,fuel_type,mw,fuel_percentage_of_total
datetime_beginning_utc,,,
2017-01-01 05:00:00,Coal,33866.5,0.41
2017-01-01 05:00:00,Gas,10516.4,0.13
2017-01-01 05:00:00,Oil,175.7,0.00
2017-01-01 06:00:00,Coal,33164.4,0.41
2017-01-01 06:00:00,Gas,10203.0,0.13


In [15]:
# Find total fossil fuel generation at each time 
df_pjm_gen = df_pjm_gen[['mw']]
df_pjm_gen = df_pjm_gen.sum(level='datetime_beginning_utc')
df_pjm_gen.head()

,mw
datetime_beginning_utc,
2017-01-01 05:00:00,44558.6
2017-01-01 06:00:00,43544.7
2017-01-01 07:00:00,42086.2
2017-01-01 08:00:00,41400.0
2017-01-01 09:00:00,41424.1


In [16]:
# Get total generation data from CEMS
df_isorto = pd.read_csv("../implement-emissions-assumptions/data/formatted_data/cems_isorto.csv", index_col='DATE_UTC')
df_isorto = df_isorto.loc[df_isorto['isorto'] == 'PJM']
df_isorto = df_isorto.loc['2017-01-01':'2017-12-31']
df_isorto.head()

,isorto,gload_mwh,so2_kg,nox_kg,pm25_kg,co2_kg,so2_dam_ap2,nox_dam_ap2,pm25_dam_ap2,so2_dam_eas,nox_dam_eas,pm25_dam_eas,co2_dam
DATE_UTC,,,,,,,,,,,,,
2017-01-01 00:00:00,PJM,44536.89,29126.293990,20943.820443,4556.971692,3.380018e+07,1219018.13,143250.87,419959.73,803371.99,155062.78,400937.50,1352007.31
2017-01-01 01:00:00,PJM,43522.44,27995.556719,20138.871544,4363.650569,3.310245e+07,1182566.21,138098.60,402719.31,772161.15,149520.53,381819.50,1324098.20
2017-01-01 02:00:00,PJM,42828.89,27274.991354,19403.799399,4230.566529,3.244870e+07,1142891.16,134688.57,389884.11,751741.45,144036.20,369406.81,1297948.01
2017-01-01 03:00:00,PJM,42536.00,27072.351396,19046.430138,4200.720142,3.211489e+07,1140297.18,132821.65,390347.97,747395.45,141525.75,368723.21,1284595.73
2017-01-01 04:00:00,PJM,42399.00,25851.675904,18707.736621,4188.200989,3.207924e+07,1087919.14,129517.44,388768.51,712626.55,139261.36,368199.26,1283169.64


In [25]:
# Compare PJM's published total with CEMS published totals 
df_isorto = df_isorto[['gload_mwh']]
df_compare = pd.merge(df_isorto,df_pjm_gen, how='inner', left_index=True, right_index=True)

df_compare.columns = ['cems', 'pjm_data_miner']
df_compare.head()

,cems,pjm_data_miner
2017-01-01 05:00:00,42721.00,44558.6
2017-01-01 06:00:00,43141.95,43544.7
2017-01-01 07:00:00,44526.13,42086.2
2017-01-01 08:00:00,45176.00,41400.0
2017-01-01 09:00:00,45845.80,41424.1


In [26]:
# find difference between two columns 
df_compare = df_compare.diff()
df_compare
# Keep rows where one of the MEF estimates is negative 
# df_compare = df_compare.loc[(df_compare.index).isin(df_pjm.index)]
# df_compare

,cems,pjm_data_miner
2017-01-01 05:00:00,NaN,NaN
2017-01-01 06:00:00,420.95,-1013.9
2017-01-01 07:00:00,1384.18,-1458.5
2017-01-01 08:00:00,649.87,-686.2
2017-01-01 09:00:00,669.80,24.1
2017-01-01 10:00:00,36.20,91.6
2017-01-01 11:00:00,-531.38,503.9
2017-01-01 12:00:00,-321.62,1605.1
2017-01-01 13:00:00,-480.00,813.9
2017-01-01 14:00:00,-352.71,1019.2


### Compare with PJM's Published MEFs 

In [29]:
df_pjm = df_pjm.loc['2014-01-01':'2018-12-31']

# Keep only MEF cols 
df_pjm = df_pjm[mef_columns]

# convert to lbs/MWh
LBS_CONVERSION = 2.20462
df_pjm = df_pjm.applymap(lambda x: x * LBS_CONVERSION)

# Aggregate over month
df_pjm['year'] = pd.DatetimeIndex(df_pjm.reset_index().DATE_UTC).year
df_pjm['month'] = pd.DatetimeIndex(df_pjm.reset_index().DATE_UTC).month
group_by = df_pjm.groupby([(df_pjm['year']),(df_pjm['month'])]).mean()

group_by

co2_mef    so2_mef    nox_mef  pm25_mef  year  month
year month                                                          
2014 1      1526.791105   0.104546   0.929027  0.103860  2014      1
     2      1399.334895   2.963419   0.969641  0.191083  2014      2
     3      1810.982844   1.176831   1.906775  0.205165  2014      3
     4      3020.689142   2.544008   6.122572  0.115926  2014      4
     5      1164.665280   0.980404   0.522258  0.096738  2014      5
     6      1369.223151   1.845258   1.093549  0.183488  2014      6
     7      1535.536745   1.515477   0.608796  0.188721  2014      7
     8      3172.059165  20.867400   4.101899  0.444553  2014      8
     9      -460.853515 -15.426093  -3.523708  0.137173  2014      9
     10     1446.063315   1.271309  -0.723539  0.187407  2014     10
     11     1295.247289   2.337195   1.859900  0.194351  2014     11
     12     1134.068356   1.254760   0.694213  0.164208  2014     12
2015 1       314.181760  -8.288731  -1.650656  0.067754  2015      1
     2      1440.566736   1.018768   1.907891  0.185408  2015      2
     3      1311.804670   2.287240   0.670177  0.217672  2015      3
     4      1243.201854   4.324284   2.494556  0.105084  2015      4
     5      1640.443759   3.643823   1.785061  0.153810  2015      5
     6      1759.582216   1.086843   1.185554  0.247322  2015      6
     7      1468.456542   2.172488   1.165275  0.189071  2015      7
     8      1535.103368   2.327916   1.547473  0.179614  2015      8
     9      1407.528020   0.689811   1.195896  0.181242  2015      9
     10     1473.409827   3.077391   1.176273  0.191391  2015     10
     11     1578.233533   2.240344   1.626720  0.236412  2015     11
     12     1786.078821   5.345582   1.718303  0.167906  2015     12
2016 1       978.627316   7.101059  -1.460306  0.653520  2016      1
     2      1481.088751   1.869524   1.208540  0.178551  2016      2
     3      1465.664497   3.257035   1.684128  0.230631  2016      3
     4      1591.090534   1.630720   1.262717  0.172445  2016      4
     5      1369.826716   2.559689   1.091826  0.161224  2016      5
     6      1375.709084   1.451423   1.063560  0.172035  2016      6
     7      1794.625334   1.035141  -0.428058  0.199059  2016      7
     8      1506.097678   1.170960   0.542660  0.173146  2016      8
     9      1455.323289   0.831100   0.623468  0.170381  2016      9
     10     1443.049843   0.939655   1.274292  0.196396  2016     10
     11     1634.593973   2.318669   2.007865  0.205137  2016     11
     12     3726.226522   5.365762  21.873778  0.475865  2016     12
2017 1     -9622.885596 -18.864729 -48.854460 -1.439926  2017      1
     2      1953.326312   2.564967   2.877617  0.253084  2017      2
     3      1775.812385  -0.753464  -0.380712  0.203142  2017      3
     4      2006.363199   2.789674   3.431942  0.356175  2017      4
     5       282.936385   4.583901  -2.558256 -0.057187  2017      5
     6      1425.220635  -1.050794  -1.180848  0.405875  2017      6
     7      1403.473441   0.903609   1.357100  0.176905  2017      7
     8       894.214811   1.935541   0.908940  0.110530  2017      8
     9      1313.875506   0.281719   0.841570  0.149258  2017      9
     10     1604.113468   1.906924   1.245366  0.192846  2017     10
     11     1481.657269  -1.811725   0.465585  0.067579  2017     11
     12     1521.346089   1.524697   0.993799  0.177402  2017     12

### Try removing values 3 standard deviations away 

In [42]:
# Remove outliers 
df_noOutliers = df[np.abs(df.co2_mef-df.co2_mef.mean()) <= (3*df.co2_mef.std())]

In [43]:
df_noOutliers[mef_columns].describe()

,co2_mef,so2_mef,nox_mef,pm25_mef
count,735764.000000,735764.000000,735764.000000,735764.000000
mean,577.061642,0.907487,0.482052,0.073546
std,2355.481363,46.670653,13.896778,1.055602
min,-115448.716549,-11145.934779,-3306.603601,-301.099021
25%,418.476288,0.001891,0.039206,0.038695
50%,557.011366,0.402649,0.344187,0.058396
75%,738.026919,1.511223,0.799959,0.094498
max,115333.452999,30612.637749,5280.981714,235.212304


In [44]:
# Find monthly MEF for PJM (to compare with their published data )
df_noOutlier_pjm = df_noOutliers.loc[df_noOutliers['isorto'] == 'PJM']
df_noOutlier_pjm = df_noOutlier_pjm.loc['2014-01-01':'2018-12-31']

# Keep MEF columns only
df_noOutlier_pjm = df_noOutlier_pjm[mef_columns]

# convert to lbs/MWh
LBS_CONVERSION = 2.20462
df_noOutlier_pjm = df_noOutlier_pjm.applymap(lambda x: x * LBS_CONVERSION)

# Year and month 
df_noOutlier_pjm['year'] = pd.DatetimeIndex(df_noOutlier_pjm.reset_index().DATE_UTC).year
df_noOutlier_pjm['month'] = pd.DatetimeIndex(df_noOutlier_pjm.reset_index().DATE_UTC).month

# Aggregate over months 
group_by = df_noOutlier_pjm.groupby([(df_noOutlier_pjm['year']),(df_noOutlier_pjm['month'])]).mean()

group_by

co2_mef   so2_mef   nox_mef  pm25_mef  year  month
year month                                                        
2014 1      1526.791105  0.104546  0.929027  0.103860  2014      1
     2      1399.334895  2.963419  0.969641  0.191083  2014      2
     3      1810.982844  1.176831  1.906775  0.205165  2014      3
     4      1770.534970  3.066526  2.090967  0.199536  2014      4
     5      1741.108905  4.139497  2.010057  0.226776  2014      5
     6      1369.223151  1.845258  1.093549  0.183488  2014      6
     7      1535.536745  1.515477  0.608796  0.188721  2014      7
     8      1565.428128  2.597310  1.617265  0.195869  2014      8
     9      1715.213633  2.967620  1.038425  0.228399  2014      9
     10     1446.063315  1.271309 -0.723539  0.187407  2014     10
     11     1295.247289  2.337195  1.859900  0.194351  2014     11
     12     1134.068356  1.254760  0.694213  0.164208  2014     12
2015 1       973.334972 -1.579004  0.526742  0.178990  2015      1
     2      1440.566736  1.018768  1.907891  0.185408  2015      2
     3      1311.804670  2.287240  0.670177  0.217672  2015      3
     4      1243.201854  4.324284  2.494556  0.105084  2015      4
     5      1640.443759  3.643823  1.785061  0.153810  2015      5
     6      1759.582216  1.086843  1.185554  0.247322  2015      6
     7      1468.456542  2.172488  1.165275  0.189071  2015      7
     8      1535.103368  2.327916  1.547473  0.179614  2015      8
     9      1407.528020  0.689811  1.195896  0.181242  2015      9
     10     1473.409827  3.077391  1.176273  0.191391  2015     10
     11     1578.233533  2.240344  1.626720  0.236412  2015     11
     12     1786.078821  5.345582  1.718303  0.167906  2015     12
2016 1      1728.249776  4.271104  1.232064  0.230844  2016      1
     2      1481.088751  1.869524  1.208540  0.178551  2016      2
     3      1465.664497  3.257035  1.684128  0.230631  2016      3
     4      1591.090534  1.630720  1.262717  0.172445  2016      4
     5      1369.826716  2.559689  1.091826  0.161224  2016      5
     6      1375.709084  1.451423  1.063560  0.172035  2016      6
     7      1437.552476  1.339942  1.026133  0.167026  2016      7
     8      1506.097678  1.170960  0.542660  0.173146  2016      8
     9      1455.323289  0.831100  0.623468  0.170381  2016      9
     10     1443.049843  0.939655  1.274292  0.196396  2016     10
     11     1634.593973  2.318669  2.007865  0.205137  2016     11
     12      978.095067  2.722659  0.832742  0.190072  2016     12
2017 1      1262.441963  0.606573  0.598165  0.146291  2017      1
     2      1417.431020  0.846484  1.536007  0.212214  2017      2
     3      1061.537742  2.146296  0.390316  0.112370  2017      3
     4      1476.790846  1.061021  2.522833  0.281566  2017      4
     5      1508.476401  2.279564  1.006019  0.150690  2017      5
     6      1425.220635 -1.050794 -1.180848  0.405875  2017      6
     7      1403.473441  0.903609  1.357100  0.176905  2017      7
     8      1422.270186  1.064136  0.568582  0.176938  2017      8
     9      1313.875506  0.281719  0.841570  0.149258  2017      9
     10     1604.113468  1.906924  1.245366  0.192846  2017     10
     11     1481.657269 -1.811725  0.465585  0.067579  2017     11
     12     1521.346089  1.524697  0.993799  0.177402  2017     12